# Private School Universe Survey

In [1]:
from collections import OrderedDict
from imp import reload
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from IPython.display import display

from hud_geo_conversions import read_zips_to_fips
from map_maker import draw_county_data_svg
from utilities import (display_cb, is_outlier_instance, is_outlier_val,
                       read_fips_codes)
from probplots import ProbPlots, plot_dists_pps

from jewish_county_data import read_jdata_counties, clean_jdata_county

pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline

SEED = 42
random.seed(SEED)

In [2]:
DATA_DIR    = '../Data/'
IMG_DIR = '../Images/'
DEM_DIR = ''.join([DATA_DIR, 'Demography/'])
SCL_DIR = ''.join([DATA_DIR, 'Schools/'])


zips_to_fips = read_zips_to_fips()  # used for multiple datasets later
fips_codes = read_fips_codes(''.join([DATA_DIR, 'Census-2010-County-FIPS.txt']))

REL_DIR  = ''.join([DATA_DIR, 'Religion/'])
RELCEN_CB_PATH  = ''.join([REL_DIR, 'US-Religion-Census-2010-County-File-CB.TXT'])
RELCEN_DAT_PATH = ''.join([REL_DIR, 'US-Religion-Census-2010-County-File.DTA'])

JDATA_PATH = ''.join([SCL_DIR, 'jdata_directory.json'])

AJPP_PATH      = ''.join([DEM_DIR, 'AJPP_County2015.xlsx'])
AJPP_GEO_PATH  = ''.join([DEM_DIR, 'AJPP_CountyGroupDefinitions2015.xlsx'])

In [3]:
SCL_DIR  = ''.join([DATA_DIR, 'Schools/'])
PSS_RAW_DICT_PATH = ''.join([SCL_DIR, 'pss_raw_dict.csv'])
PSS_RAW_PATH = ''.join([SCL_DIR, 'pss1112_pu.txt'])

In [4]:
from pss import read_pss_raw_dict, read_pss_raw

pss_raw_dict_all = read_pss_raw_dict(PSS_RAW_DICT_PATH)

In [5]:
are_resps = lambda df: df.Code.str.startswith('Q')
pss_raw_dict_qs = pss_raw_dict_all.loc[are_resps, :]
pss_raw_dict_qs.head()

,Pos,Var_Type,Len,Code,Label,QNum
Name,,,,,,
P135,91,NUM,8,Q5A_GRD,5A Ungraded Offered,5
P140,92,NUM,8,Q5A_ENRL,5A Ungraded Enrollment,5
P145,93,NUM,8,Q5B_GRD,5B Nursery/Prekindergarten Offered,5
P150,94,NUM,8,Q5B_ENRL,5B Nursery/Prekindergarten Enrollment,5
P155,95,NUM,8,Q5C_GRD,5C Kindergarten Offered,5


In [6]:
pss_raw = read_pss_raw(PSS_RAW_PATH)
pss_raw.head()


,PFNLWT,REPW1,REPW2,REPW3,REPW4,REPW5,REPW6,REPW7,REPW8,REPW9,...,F_P660,S_KG,P_INDIAN,P_ASIAN,P_PACIFIC,P_HISP,P_WHITE,P_BLACK,P_TR,STTCH_RT
ID,,,,,,,,,,,,,,,,,,,,,
A1192001,1.084,1.084,1.084,1.084,1.084,1.084,1.084,1.084,1.084,1.084,...,0,6,0.000,0.000,0.000,3.448,1.724,91.379,3.448,6.988
A1192003,1.049,1.049,1.049,1.049,1.049,1.049,1.049,1.049,1.049,1.049,...,0,14,0.000,0.000,0.000,0.000,0.000,100.000,0.000,6.869
BB120001,1.320,2.640,0.000,0.000,2.640,0.000,0.000,2.640,2.640,0.000,...,0,22,0.000,0.000,0.000,31.818,0.000,68.182,0.000,36.667
A0500002,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,0,2,0.000,0.000,0.000,0.000,91.111,8.889,0.000,11.250
00000769,1.147,1.147,1.147,1.147,1.147,1.147,1.147,1.147,1.147,1.147,...,0,45,0.000,1.107,0.000,1.476,82.657,12.177,2.583,10.586


In [7]:
from pss import read_pss_table

PSS_PATH = ''.join([SCL_DIR, 'ELSI_csv_export_6362018465286175957399.csv'])
pss, pss_header, pss_footer = read_pss_table(PSS_PATH)
pss = (pss.loc[pss.Religion=='Jewish', :]
         .drop(['Religion', 'Orientation'], axis=1)
)
print('No. Jewish schools: {}'.format(pss.shape[0]))
pss.State.value_counts().head()
pss.head()

No. Jewish schools: 793


,Name_shorter,State,State2,FIPS_State,Name,County,FIPS,Days,Hours,Library,...,Black_No,Black_Pc,White_No,White_Pc,Hawaiian_No,Hawaiian_Pc,Multi_No,Multi_Pc,Student_Teach_Ratio,Teachers_FTE
ID,,,,,,,,,,,,,,,,,,,,,
02157581,ABRAHAM JOSHUA HESCHEL DAY SCH,CA,CALIFORNIA,06,ABRAHAM JOSHUA HESCHEL DAY SCHOOL,LOS ANGELES,06037,180.000,7.500,1-Yes,...,nan,nan,nan,nan,nan,nan,nan,nan,7.680,52.500
A9104665,ABRAHAM JOSHUA HESCHEL HIGH SC,NY,NEW YORK,36,ABRAHAM JOSHUA HESCHEL HIGH SCHOOL,NEW YORK,36061,167.000,8.750,1-Yes,...,0.000,0.000,271.000,99.270,0.000,0.000,0.000,0.000,4.840,56.400
A9100497,ADAT ARI EL DAY SCHOOL,CA,CALIFORNIA,06,ADAT ARI EL DAY SCHOOL,LOS ANGELES,06037,176.000,6.170,1-Yes,...,0.000,0.000,184.000,98.920,0.000,0.000,0.000,0.000,6.410,29.000
BB120051,ADAT SHALOM,CA,CALIFORNIA,06,ADAT SHALOM,LOS ANGELES,06037,nan,nan,2-No,...,0.000,0.000,55.000,90.160,0.000,0.000,3.000,4.920,12.200,5.000
01263546,ADDLESTONE HEBREW ACADEMY,SC,SOUTH CAROLINA,45,ADDLESTONE HEBREW ACADEMY,CHARLESTON,45019,178.000,7.420,1-Yes,...,2.000,1.960,100.000,98.040,0.000,0.000,0.000,0.000,4.230,24.100


In [8]:
SCL_DIR  = ''.join([DATA_DIR, 'Schools/'])
PSS_RAW_DICT_PATH = ''.join([SCL_DIR, 'pss_raw_dict.csv'])
PSS_RAW_PATH = ''.join([SCL_DIR, 'pss1112_pu.txt'])

# JData vs PSS

Leave PSS data since no significant number missing from JData

In [9]:
import jdata
jd_df = (jdata.read_jdata(JDATA_PATH)
         .pipe(jdata.extract_clean_jdata))

In [10]:
jd_df.shape, pss.shape

((3821, 10), (793, 60))

In [11]:
def norm_addr(cols):
    """Removes abbreviation points and converts to upper case."""
    
    cols = cols.apply(lambda x: x.str.replace('.', '').str.upper())
    return cols

def make_addr_key(addr_col):
    """Creates column from address no. and first part of name."""
    
    po_box = r'^(?:PO BOX \d+)'
    others = r'^(?:(?:\d+[ ])?[^ ]+)'
    pat = r''.join(['(', po_box, '|', others, ')'])
    key_col = addr_col.str.extract(pat, expand=False)
    
    # Standardize North, South, East, West
    convert = {r'[ ]NORTH':' N',
               r'[ ]SOUTH':' S',
               r'[ ]EAST':' E',
               r'[ ]WEST':' W'}
    key_col = key_col.replace(convert, regex=True)
    
    return key_col

def make_name_key(col):
    """Creates column for first part of name for merge."""
    
    col = col.str.extract(r'([^ ]+)')
    return col

jd_normed = jd_df.copy()
upper_cols=['Name', 'Addr', 'City']
jd_normed[upper_cols] = norm_addr(jd_normed[upper_cols])
pss[upper_cols] = norm_addr(pss[upper_cols])

jd_normed['Addr_Key'] = make_addr_key(jd_normed.Addr)
pss['Addr_Key'] = make_addr_key(pss.Addr)
jd_pss_1 = pd.merge(pss.reset_index(), jd_normed,
                    on=['Addr_Key', 'City', 'State'])

jd_normed['Name_Key'] = make_addr_key(jd_normed.Name)
pss['Name_Key'] = make_addr_key(pss.Name)
jd_pss_2 = pd.merge(pss.reset_index(), jd_normed,
                    on=['Name_Key', 'City', 'State'])

in_both = (pss.index.isin(jd_pss_1.ID)) | (pss.index.isin(jd_pss_2.ID))
pss_only = pss[~in_both]
print(pss_only.shape)

(106, 62)


# Extensive Cleaning/Exploration

In [12]:
import sys
import itertools as it
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from collections import OrderedDict
import re
import pandas.util.testing as tm
import pdb
from glob import glob
import logging
from scipy.stats import distributions, probplot
# tm.assert_frame_equal(result, X)

from probplots import ProbPlots
from pss import *

%matplotlib inline

# Make the graphs a bit prettier, and bigger
plt.style.use('default')
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['font.family'] = 'sans-serif'

pd.set_option('display.multi_sparse', True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


# TODO: explore pandas options

SEED = 42

def display_cb(df):
    """Displays Dataframe with long descriptions.
    
    Used for codebooks.
    """
    with pd.option_context('max_colwidth', 1000):
        display(df)
        
DATA_DIR    = '../Data/'
SCRAPES_DIR = '../Scrapes'

SCL_DIR  = ''.join([DATA_DIR, 'Schools/'])
REL_DIR  = ''.join([DATA_DIR, 'Religion/'])
DEM_DIR  = ''.join([DATA_DIR, 'Demography/'])

TS_DIR = '/Users/mdlynch37/projects/coding/learning-math-and-stats/ThinkStats2/code'
sys.path.append(TS_DIR)
import thinkstats2
import thinkplot

### Raw data

In [13]:
PSS_RAW_DICT_FP = ''.join([SCL_DIR, 'pss_raw_dict.csv'])
PSS_RAW_FP = ''.join([SCL_DIR, 'pss1112_pu.txt'])

In [14]:
pss_raw_dict_all = read_pss_raw_dict(PSS_RAW_DICT_FP)
          
are_resps = lambda df: df.Code.str.startswith('Q')
pss_raw_dict_qs = pss_raw_dict_all.loc[are_resps, :]
pss_raw_dict_qs.head()

,Pos,Var_Type,Len,Code,Label,QNum
Name,,,,,,
P135,91,NUM,8,Q5A_GRD,5A Ungraded Offered,5
P140,92,NUM,8,Q5A_ENRL,5A Ungraded Enrollment,5
P145,93,NUM,8,Q5B_GRD,5B Nursery/Prekindergarten Offered,5
P150,94,NUM,8,Q5B_ENRL,5B Nursery/Prekindergarten Enrollment,5
P155,95,NUM,8,Q5C_GRD,5C Kindergarten Offered,5


In [15]:
pss_raw = read_pss_raw(PSS_RAW_FP)
pss_raw.head()

,PFNLWT,REPW1,REPW2,REPW3,REPW4,REPW5,REPW6,REPW7,REPW8,REPW9,...,F_P660,S_KG,P_INDIAN,P_ASIAN,P_PACIFIC,P_HISP,P_WHITE,P_BLACK,P_TR,STTCH_RT
ID,,,,,,,,,,,,,,,,,,,,,
A1192001,1.084,1.084,1.084,1.084,1.084,1.084,1.084,1.084,1.084,1.084,...,0,6,0.000,0.000,0.000,3.448,1.724,91.379,3.448,6.988
A1192003,1.049,1.049,1.049,1.049,1.049,1.049,1.049,1.049,1.049,1.049,...,0,14,0.000,0.000,0.000,0.000,0.000,100.000,0.000,6.869
BB120001,1.320,2.640,0.000,0.000,2.640,0.000,0.000,2.640,2.640,0.000,...,0,22,0.000,0.000,0.000,31.818,0.000,68.182,0.000,36.667
A0500002,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,0,2,0.000,0.000,0.000,0.000,91.111,8.889,0.000,11.250
00000769,1.147,1.147,1.147,1.147,1.147,1.147,1.147,1.147,1.147,1.147,...,0,45,0.000,1.107,0.000,1.476,82.657,12.177,2.583,10.586


### Table from online generator

__Data retrieved from [NCES Table Generator][NCESTableGenerator] in csv format__

- 2011-2012 data saved with generator, retrievable with Table ID 54242 from generator start page
- 2009-2010 data saved with generator to Table ID 54245
- All available fields selected for All 50 States + DC + Outlying Areas

[NCESTableGenerator]:https://nces.ed.gov/ccd/elsi/tableGenerator.aspx

In [16]:
PSS_FP = ''.join([SCL_DIR, 'ELSI_csv_export_6362018465286175957399.csv'])


def list_keys(odict):
        return list(odict.keys())

def list_vals(odict):
        return list(odict.values())
    
def inverse_dict(d):
    return {val:key for key, val in d.items()}

def add_to_pss_from_raw(df_table, df_raw, cols_odict, cols_to_add, codes_to_vals=None, append=True):
    """Include cols from raw data in table.
    
    Parameters
    ----------
    df_table : DataFrame
    df_raw : DataFrame
    cols_to_add : dict
        New names mapped to originals selected to merge
    codes_to_vals : dict mapping to dict
        Specifies which cols (new names) with coded vals
        to convert to meaningful vals like:
        {col_name:{code:s}}
        
    Returns
    -------
    df : DataFrame, resulting table
    """
    
    cols_odict.update(cols_to_add)
    left, right = df_table, df_raw[list_vals(cols_to_add)]
    if not append:
        left, right = right, left
        for key in reversed(cols_to_add.keys()):
            cols_odict.move_to_end(key, last=False)
        
    df = (pd.merge(left, right, left_index=True, right_index=True)
          .rename(columns=inverse_dict(cols_to_add))
         )
    if codes_to_vals:
        df = df.replace(codes_to_vals)

    return df, cols_odict

cols_to_add_left = OrderedDict([
    ('Interview_Meth', 'CSOURCE'),
    ('To_College', 'P360')
])

cols_to_add_right = OrderedDict([
    ('No Associations', 'P450'),
    ('National Society of Hebrew Day Schools', 'P515'),
    ('The Jewish Community Day School Network', 'P522'),
    ('Solomon Schechter Day Schools', 'P525'),
    ('National Assn of Independent Schools', 'P620')
])
codes_to_vals = {
    'Interview_Meth':{
        1:'Mail',
        2:'Internet',
        3:'Regional Office follow-up',
        4:'Robocall'  # computer-assisted telephone interview
    }
}


# pss_df, pss_header, pss_footer = read_pss_table(PSS_FP)
# pss_df, pss_cols = add_to_pss_from_raw(pss_df, pss_raw,
#                                        cols_to_add_left, codes_to_vals, append=False)
# pss_df, pss_cols = add_to_pss_from_raw(pss_df, pss_raw,
#                                        cols_to_add_right, append=True)

In [17]:
def drop_pss_cols(df_table, cols_odict, cols):
    df_table = df_table.drop(cols, axis=1)
    for col in cols:
        cols_odict.pop(col)
    
    return df_table, cols_odict
    
cols_to_drop = {
    'Name_shorter',
    'State2',
    'State3',
    'Total_Students_Excl_PK',
    'Zip4',
    'Zip_Full',
    'Phone',
    'Lowest_Gr',
    'Highest_Gr'
}

pss_df, pss_cols = drop_pss_cols(pss_df, pss_cols, cols_to_drop)

print(pss_header)
display(pss_df.shape)
display(pss_df.head(3))
print(pss_footer)

NameError: name 'pss_df' is not defined

Reorder cols...

Get target population...

In [ ]:
pss_j = (pss_df.loc[pss_df.Religion=='Jewish', :]
         .drop(['Religion', 'Orientation'], axis=1)
)
pss_raw_j = pss_raw.loc[pss_raw['P440']==18, :]

print('No. Jewish schools: {}'.format(pss_j.shape[0]))
print('No. states with Jewish schools: {}'.format(len(pss_j.State.unique())))
display(pss_j.State.value_counts().head())

#### Add and Clean Association memberships

In [ ]:
def plt_assc_overlap(assc_lbls, df_raw, df_raw_dict):

    assc_data = (df_raw.loc[:, assc_lbls]
                .rename(columns=
                        df_raw_dict.Label.str.strip('15 ').to_dict())
                )

    nrows  = int(-(-assc_data.shape[1]//2))  # quick ciel division
#     pdb.set_trace()
    fig, axes = plt.subplots(nrows=nrows, ncols=2, figsize=(15, 50), sharex=True, sharey=True)

    def zero_assc(ser, assc):
        ser[assc] = 0
        return ser
    
    for i, assc in enumerate(assc_data.columns):
        _ = (assc_data.loc[assc_data[assc]==1, :]
             .sum()
             .pipe(zero_assc, assc)
             .plot(kind='barh', ax=axes[i//2][i%2], title=''.join(
                 [assc, ' (', str(assc_data[assc].sum()), ')']))
            )
    plt.tight_layout()
    plt.show()


assc_qs = pss_raw_dict_qs.loc[pss_raw_dict_qs.QNum=='15', :]
cnt_asscs = (pss_raw_j
             .loc[:, assc_qs.index]
             .sum(0)
             .rename('N'))

assc_ls = (assc_qs[cnt_asscs > 0]
           .join(cnt_asscs)
           .sort_values('N', ascending=False))

if False:
    plt_assc_overlap(assc_ls.index, pss_raw_j, pss_raw_dict_qs)

#### Clean `Locale` and  `Community` 

In [ ]:
def to_categorical(col):
    
    if not col.dtype.name == 'category':
        col = (col.str.extract(r'\d+-(\w+)', expand=False)
               .astype('category'))
        
    return col

In [ ]:
pss_j['Community'] = to_categorical(pss_j['Community'])

display(pss_j.Community.value_counts(dropna=False))
display(pss_j.Locale.value_counts(dropna=False))

assert not pss_j.Community.isnull().any(), 'Community vals need filling!'
assert not pss_j.Locale.isnull().any(), 'Locale vals need filling!'

pss_j = pss_j.drop('Locale', axis=1)

#### Clean `Levels`, `Type_` and `Coed`

In [ ]:
pss_j['Level']  = to_categorical(pss_j['Level'])
pss_j.Level.value_counts(dropna=True)

In [ ]:
# imputed by inspection
fills = {
    'Level': {
        'A9104292':'Elementary',
        '00087434':'Elementary',
        '01900102':'Combined',
        'A0108086':'Elementary'
    },
    'Type_': {
        'A0501613':'1-Regular Elementary or Secondary',
        'A1101672':'1-Regular Elementary or Secondary'
    },
    'Coed': {
        'BB121609':'1-Coed (school has male and female students)',
        'BB040270':'1-Coed (school has male and female students)'
    }
        }
         
for feat, vals in fills.items():
    for ID, val in vals.items():
        pss_j.loc[ID, feat] = val

have_null = pss_j.loc[:, ['Level', 'Type_', 'Coed']].isnull().any(1)
assert not have_null.any(), 'Fills for Level, Type_ and Coed not complete!'

#### Remove duplicate schools

In [ ]:
# check dupes, same address, city, state
pss_dupe_addr = pss_j[pss_j[['Addr', 'City', 'County', 'State']]
                      .duplicated(keep=False)]
print('{} schools share the same address, before deduping.'.format(pss_dupe_addr.shape[0]))

# Display for manual inspection
if False:
    display(pss_dupe_addr.sort_values(
        ['State', 'County', 'City', 'Addr']).loc[:,:].T
           )
# clearly duplicate from looking at different defs of dupes
dupes_to_drop = [
    'BB980701', 'BB121352', 'BB121671', 'A0302192', 'A9104913', 'BB121497', 
    'BB121621', 'BB000817', 'A0302192', 'BB121831', 'BB121771'
]

pss_j = pss_j.drop(dupes_to_drop)
print('{} schools share the same address, after first deduping.'.format(pss_dupe_addr.shape[0] - len(dupes_to_drop)))

In [ ]:
# check dupes, same name different address only
pss_dupe_name = pss_j[pss_j[['Name', 'City']].duplicated(keep=False)]
print('{} schools share the name and state.'.format(pss_dupe_name.shape[0]))

if False:
    display(pss_dupe_name.sort_values(['Name', 'City']).loc[:,:].T)

#### Clean Student Counts

In [ ]:
pss_j.columns

In [ ]:
# numbers add up for subtotals
df_temp = pss_j.loc[:, 'Total_Students':'12_Gr'].fillna(0)
assert (df_temp['PK-K'] == df_temp.loc[:, 'PK_Gr':'K_Gr'].sum(1)).all()
assert (df_temp['Gr1-8'] == df_temp.loc[:, '1_Gr':'8_Gr'].sum(1)).all()
assert (df_temp['Gr9-12'] == df_temp.loc[:, '9_Gr':'12_Gr'].sum(1)).all()

In [ ]:
sub_gr_totals   = pss_j.loc[:, 'PK-K':'Gr9-12'].sum(1)
valid_gr_tot    = sub_gr_totals == pss_j.Total_Students

print('{} records with grade inconsistencies.'
      .format(pss_j[~valid_gr_tot].shape[0]))

In [ ]:
sub_gr_totals['00087434']

In [ ]:
if False:
    display(pss_j[~valid_gr_tot].T)

In [ ]:
# p = pss_j.copy()

In [ ]:
# switched to False when no need to inspect
if False:
    
    df = pss_j[~valid_gr_tot]
    diff_col = 'diff------------>'

    parts = [df.loc[:, :'Total_Students']]
    parts.append(pd.DataFrame(sub_gr_totals[~valid_gr_tot], 
                              columns=['Sub_Gr_Totals']))
    parts.append(pd.DataFrame(df.Total_Students - sub_gr_totals[~valid_gr_tot], 
                              columns=[diff_col]))
    parts.append(df.loc[:, 'PK-K':'12_Gr'])

    display(pd.concat(parts, axis=1, verify_integrity=True
                     ).sort_values(diff_col, ascending=True).T)
    
    
# fill empty grand totals with sum of sub totals
pss_j['Total_Students'] = \
        pss_j.Total_Students.mask(lambda x: (x==0) | (x.isnull()), sub_gr_totals)

In [ ]:
pss_j = pss_j.drop(pss_j.loc[:, 'PK-K':'Ungraded'].columns, axis=1)

In [ ]:
pss_j.columns

In [ ]:
probplots = ProbPlots(pss_j.Total_Students)
probplots.plot_probplot(dist_name='expon')

In [ ]:
probplots.plot_probplot(dist_name='lognorm')

In [ ]:
probplots = ProbPlots(np.log(pss_j.Total_Students))
probplots.plot_probplot(dist_name='norm')

#### Clean Race Counts

In [ ]:
def sum_non_white_cols(df, insert_before='Amer_Ind_No'):
    """Return DataFrame with non-white aggregate columns.
    
    Parameters
    ----------
    df : DataFrame
    insert_before : str
        Specifies where to insert new columns.

    """
    # in case function has been called prior
    df = df.loc[:, ~df.columns.isin(['NonWhite_No', 'NonWhite_Pc'])]
    
    cols = df.columns.sort_values()
    Ps = [col for col in cols if col.endswith('_Pc') and col!='White_Pc']
    Ns = [col for col in cols if col.endswith('_No') and col!='White_No']
    
    assert all([n[:-2]==p[:-2] for n, p in zip(Ps, Ns)]), (
            'Name or number mismatch between _No cols and _Pc cols.')

    nw_no    = df.loc[:, Ns].sum(1)
    nw_perc =  nw_no/df.Total_Students*100 
    
    new_cols = pd.DataFrame(
        [nw_no, nw_perc], index=['NonWhite_No', 'NonWhite_Pc']).T

    idx_insert = df.columns.get_loc(insert_before)
    df = pd.concat([
        df.iloc[:, :idx_insert],
        new_cols,
        df.iloc[:, idx_insert:]], axis=1)
    
    return df


In [ ]:
pss_j = sum_non_white_cols(pss_j)

r_detail_tots = pss_j.NonWhite_No+pss_j.White_No

print('{} records have no race data.'
      .format((r_detail_tots==0).sum()))
print('{} schools have data on non-white students.'
      .format((pss_j.NonWhite_No>0).sum()))
print('')
print('{} records with valid race breakdowns.'
      .format((r_detail_tots==pss_j.Total_Students).sum()))
print('{} of these have nonwhite race data.'
      .format(((r_detail_tots==pss_j.Total_Students) & 
               (pss_j.NonWhite_No>0)).sum()))
print('')
print('{} records with only white race data.'
      .format(((pss_j.NonWhite_No==0) & (pss_j.White_No>0)).sum()))
print('{} records with only nonwhite race data.'
      .format(((pss_j.NonWhite_No>0) & (pss_j.White_No==0)).sum()))
print('')
print('{} records with race sum inconsistencies.'
      .format((pss_j.NonWhite_No+pss_j.White_No!=pss_j.Total_Students).sum()))


if False:
    display(pss_j.sort_values('NonWhite_Pc', ascending=False).iloc[20:, :].T)  


In [ ]:
pss_j = sum_non_white_cols(pss_j)

r_detail_tots = pss_j.NonWhite_No+pss_j.White_No

print('{} records have no race data.'
      .format((r_detail_tots==0).sum()))
print('{} schools have data on non-white students.'
      .format((pss_j.NonWhite_No>0).sum()))
print('')
print('{} records with valid race breakdowns.'
      .format((r_detail_tots==pss_j.Total_Students).sum()))
print('{} of these have nonwhite race data.'
      .format(((r_detail_tots==pss_j.Total_Students) & 
               (pss_j.NonWhite_No>0)).sum()))
print('')
print('{} records with only white race data.'
      .format(((pss_j.NonWhite_No==0) & (pss_j.White_No>0)).sum()))
print('{} records with only nonwhite race data.'
      .format(((pss_j.NonWhite_No>0) & (pss_j.White_No==0)).sum()))
print('')
print('{} records with race sum inconsistencies.'
      .format((pss_j.NonWhite_No+pss_j.White_No!=pss_j.Total_Students).sum()))


if False:
    display(pss_j.sort_values('NonWhite_Pc', ascending=False).iloc[20:, :].T)  


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18,5))

_ = (pss_j.loc[:, ['NonWhite_Pc']]
     .assign(Bdown_Tot_Pc=(pss_j.NonWhite_No+pss_j.White_No)/pss_j.Total_Students*100)
     .sort_values('NonWhite_Pc')
     .sort_values('Bdown_Tot_Pc', kind='mergesort')  # for stable sort
     .plot(ax=ax1, title='% of total students')
    )

_ = (pss_j.loc[:, ['NonWhite_Pc', 'Total_Students']]
     .assign(tot_sub_race=pss_j.NonWhite_No+pss_j.White_No)
     .sort_values('NonWhite_Pc')
     .query('tot_sub_race == Total_Students')
     .plot(y='NonWhite_Pc', ax=ax2, title='% nonwhite in consistent records')
    )

plt.tight_layout()
plt.show()


In [ ]:
sns.distplot(np.log(pss_j.NonWhite_Pc).replace(float('-inf'), 0), hist_kws=dict(align='left'))

In [ ]:
probplots = ProbPlots(pss_j.NonWhite_Pc)
probplots.plot_probplot(dist_name='pareto')

In [ ]:
sns.distplot(pss_j.NonWhite_Pc, hist_kws=dict(align='left'))

In [ ]:
race_imps = pss_raw_j.loc[:, 'F_P320':'F_P332']
im_cols = {
    'F_P310':'Im_AmerInd', 'F_P316':'Im_Asian', 'F_P318':'Im_Hawai',
    'F_P320':'Im_Hisp', 'F_P325':'Im_Black', 'F_P330':'Im_White',
    'F_P332':'Im_Multi'}

if False:
    display(pss_j
            .merge(race_imps[race_imps.sum(1)>0],
                   left_index=True, right_index=True)
            .rename(columns=im_cols)
            .sort_values('NonWhite_No', ascending=False)
            .T)

No evidence of raw race data being changed/imputed.

In [ ]:
if False:
    display(pss_j.query('NonWhite_Pc > 10').sort_values('NonWhite_Pc', ascending=False).T)

In [ ]:
IDs_to_drop = ['00940306', 'A1192056', 'A1100834', 'A1100012']  # closed or not jewish
pss_j = pss_j.loc[~pss_j.index.isin(IDs_to_drop)]

multi_to_wh = ['00078758', 'AA890059', 'A0302252', 'A9100648']

pss_j.loc[pss_j.index.isin(multi_to_wh), 'White_No'] = \
    pss_j.loc[pss_j.index.isin(multi_to_wh), 'Multi_No']

pss_j.loc[pss_j.index.isin(multi_to_wh), 'Multi_No'] = 0

all_ind_err = ['A9105127']

pss_j.loc['A9105127', 'Amer_Ind_No':'Multi_Pc'] = 0

clear_then_imp  = [  # maybe
    'A0307781', 'A0302232', 'A1102245', '00080304', 'A1101443',
    'K9301027', '00935872', '00646064', 'BB100145']

pss_j = sum_non_white_cols(pss_j)

In [ ]:
plt.figure(figsize=(12, 8))
(pss_j[['Total_Students', 'NonWhite_No', 'NonWhite_Pc', 'Solomon Schechter Day Schools']]
 .pipe((sns.boxplot, 'data'), x='NonWhite_Pc', y='Solomon Schechter Day Schools', orient='h'))
sns.despine()

plt.tight_layout()
plt.show()

In [ ]:
probplots = ProbPlots(pss_j.NonWhite_Pc)
probplots.plot_probplot(dist_name='pareto')

Start with `Student_Totals`...

In [ ]:
# def is_outlier(col, p=1.5):
#     q3 = col.quantile(.75)
#     q1 = col.quantile(.25)
#     lo = q1 - (q3 - q1)*p
#     hi = q3 + (q3 - q1)*p
    
#     return (col < lo) | (col > hi)

# g = pss_j.groupby('Community')
# x = g.Total_Students.apply(is_outlier, p=1.5)
# # pss_j[x].T

Totals are taken care of!

### Impute missing/outlying values

Manually fill-in important fields with few instances of missing values.

In [ ]:
pss_j.Student_Teach_Ratio.describe()

In [ ]:
# cond = pss_j.Student_Teach_Ratio < 80
sns.distplot(pss_j.Student_Teach_Ratio.dropna())

In [ ]:
# pss_j.Student_Teach_Ratio = c

In [ ]:
min_ = 2
max_ = 80
col = pss_j.Student_Teach_Ratio
cond = (col<min_) | (col>max_)
pss_j.loc[cond ,'Student_Teach_Ratio'] = col.median()

In [ ]:
sns.boxplot(pss_j.Student_Teach_Ratio)

In [ ]:
pss_j.info()

### Percent to college

In [ ]:
bins = np.arange(0,101, 20)
indices = np.digitize(pss_j.To_College, bins, right=True)
groups = pss_j.To_College.groupby(indices)

groups.count()

In [ ]:
df = pss_j.replace({'To_College':{0.:np.nan}})
df.loc[df.To_College<5, 'To_College'] = np.nan
sns.boxplot(x='Type_', y='To_College', data=df[df.Level!='Elementary'])

In [ ]:
sns.distplot(df.To_College.dropna())

In [ ]:
pss_j.T

In [ ]:
# grade_feats = ['PK-K', 'Gr1-8', 'Gr9-12', 'Ungraded']
# # pss_j[grade_feats] = pss_j[grade_feats].fillna(0)

# assert (pss_j.Total_Students == pss_j[grade_feats].fillna(0).sum(1)).all()

In [ ]:
# cat_feats = ['Library', 'Locale', 'Coed', 'Type', 'Level', 'Community']
# pss_j[cat_feats] = to_categoricals(pss_j[cat_feats])

# pss_j[cat_feats] = fillna_cats(pss_j[cat_feats])